# Plantilla de minería de texto
Este cuaderno incluye código Python para descargar texto escaneado de periódicos fronterizos y realizar análisis de texto de frecuencia de palabras en los periódicos.

Para comenzar, siga las instrucciones en la sección [Configuración](#Configuración), a continuación. Una vez que sepa qué datos le gustaría usar, hay varias opciones enumeradas a continuación para diferentes análisis de texto.

El trabajo es parte de dos proyectos:
+ _Using Newspapers as Data for Collaborative Pedagogy: A Multidisciplinary Interrogation of the Borderlands in Undergraduate Classrooms_ (_El Uso de Periódicos como Datos para la Pedagogía Colaborativa: Una Interrogación Multidisciplinaria de las Fronteras en las Aulas de Pregrado_), financiado en parte por la Fundación Mellon a través de la programa [Collections as Data](https://collectionsasdata.github.io/part2whole/). Más información sobre el proyecto está disponible en
[https://libguides.library.arizona.edu/newspapers-as-data](https://libguides.library.arizona.edu/newspapers-as-data).
+ _Reporting on Race and Ethnicity in the Borderlands (1882-1924): A Data-Driven Digital Storytelling Hub_ (_Reportaje sobre Raza y Etnicidad en las Tierras Fronterizas (1882-1924): Un Centro de Narración Digital Basado en Datos_), financiado por la Fundación Mellon a través de el [Digital Borderlands](http://borderlands.digitalscholarship.library.arizona.edu/) programa.

Si no está familiarizado con la minería de datos de texto, eche un vistazo a este agradable [StoryMap](https://arcg.is/1j84jz) que presenta la idea de la minería de datos de texto y lo que podemos hacer con ella.

Este cuaderno y las lecciones adicionales de minería de texto están disponibles en [https://github.com/jcoliver/dig-coll-borderlands](https://github.com/jcoliver/dig-coll-borderlands).

## Configuración
La primera decisión que debe tomar es si desea utilizar un conjunto de datos de muestra pequeño o un conjunto de datos más grande. La última opción requiere que se descarguen los archivos y puede tardar unos minutos.

Si _no_ desea usar el conjunto más grande de texto escaneado, puede usar los datos que se distribuyen con este cuaderno. Ejecutar el bloque de código a continuación le mostrará los datos que están disponibles si no desea descargar el conjunto de datos más grande (no necesita realizar ningún paso adicional para usar los datos a continuación, vienen con este Jupyter Notebook).

In [ ]:
# Run to display table with newspaper information
import pandas
titles = pandas.read_csv('data/sample/sample-titles.csv')
datapath = 'data/sample/'
display(titles)

Si desea utilizar el conjunto completo de periódicos fronterizos escaneados, primero deberá descargar los archivos del depósito de datos de la Universidad de Arizona. Ejecutar el bloque de código a continuación hará esto por usted (si solo quiere probar cosas con un conjunto de datos más pequeño, no ejecute este bloque y simplemente avance). Tenga en cuenta que los datos están contenidos en un archivo de alrededor de 1,5 GB e incluyen cientos de miles de archivos. Tanto los pasos de descarga como los de extracción de archivos pueden demorar un poco (¿5 minutos? ¿10?), por lo que ahora podría ser un buen momento para volver a llenar su bebida. Cuando se complete el proceso de descarga y extracción, se imprimirá una tabla que muestra los datos disponibles debajo del bloque de código.

In [ ]:
# import the libraries necessary for download & extraction
from urllib.request import urlretrieve
import zipfile
import os
import pandas

# Location of the file on the UA Data Repository
url = 'https://arizona.figshare.com/ndownloader/files/31104157'

# Download the file & write it to disk
zip_filename = 'fulldata.zip'
download = urlretrieve(url, zip_filename)

# Set the destination for the data files
destination = 'data/complete/'

# Make sure the destination directory exists
if(not(os.path.isdir(destination))):
    os.makedirs(destination)

# Extract files to destination directory
with zipfile.ZipFile(zip_filename, 'r') as zipdata:
    zipdata.extractall(destination)
    
# No need for that zipfile, so we can remove it
os.remove(zip_filename)

# Finally, display the available titles for this full data set
full_titles = pandas.read_csv('data/complete/complete-titles.csv')
datapath = 'data/complete/'
display(full_titles.sort_values(by=['name']))

## Análisis de frecuencia de palabras
Para los periódicos de su elección, hay una variedad de análisis que se pueden realizar con el código a continuación.
+ [Investigar la frecuencia de palabras a lo largo del tiempo para una sola palabra (o conjunto de palabras) en un solo periódico](#Una-sola-palabra-en-un-periódico)
+ [Investigar la frecuencia de palabras a lo largo del tiempo en un par de periódicos](#Una-sola-palabra-en-dos-periódicos)
+ [Investigar frecuencias de dos palabras a lo largo del tiempo para un solo periódico](#Dos-palabras-en-un-periódico)

En todos los análisis, el siguiente código tiene valores de ejemplo para periódicos, palabras y fechas. Puede cambiarlos según sea necesario para su pregunta específica.

Sin embargo, antes de comenzar, asegúrese de ejecutar el bloque de código inmediatamente debajo, que se carga en todas las bibliotecas necesarias para los análisis de datos de texto posteriores.

In [ ]:
# No need to change anything, just run this block of code to load necessary libraries

# for data tables
import pandas

# for file navigation
import os

# for pattern matching in filenames
import re

# for text data mining
import nltk

# for stopword corpora for a variety of languages
from nltk.corpus import stopwords

# for splitting data into individual words
from nltk.tokenize import RegexpTokenizer

# for automated text cleaning
import digcol as dc

# download the stopwords for several languages
nltk.download('stopwords')

# for drawing the plot
import plotly.express as px

### Una sola palabra en un periódico
El siguiente código está diseñado para analizar un conjunto de palabras para un título de periódico individual. Tal como está escrito, el bloque de código observará la frecuencia de las palabras relacionadas con la influenza ("gripe", "influenza") en _The Bisbee Daily Review_ durante los años 1917 y 1918.

Puede editar los valores de `título`, `lista_años`, `mis_palabras` e `idioma` para que se ajusten a su análisis de interés. Para el valor de `título`, asegúrese de utilizar el valor de la columna "directorio" de la tabla anterior que corresponda al periódico de interés. Por ejemplo, si quisieras mirar _El Tucsonense_, cambia esto:

`title = 'revisión-diaria-de-bisbee'`

a esto:

`title = 'el-tucsonense'`

Para `year_list`, enumere todos los años de interés, cada uno entre comillas simples (') y valores separados por una coma. Si solo está interesado en un año, no es necesaria la coma.

Las palabras listadas en `mis_palabras` serán efectivamente "agrupadas"; es decir, para este ejemplo, la gráfica mostrará la frecuencia de 'gripe' e 'influenza' combinadas. Si está interesado en trazar conjuntos de palabras _separados_, consulte la sección [Investigar frecuencias de dos palabras a lo largo del tiempo para un solo periódico](#Dos-palabras-en-un-periódico), a continuación.

Finalmente, asegúrese de que el valor de 'idioma' corresponda al idioma del periódico que está viendo (consulte la tabla en la parte superior de la página para obtener información sobre el idioma. Tenga en cuenta que el valor debe estar en minúsculas; es decir, use 'español' _no_ 'Español'.

In [ ]:
# Code for one set of words, one newspaper

# Include 'flu' in words to look for
title = 'bisbee-daily-review'     # Make sure this matches "directory" in table above
year_list = ['1917', '1918']      # Each item is separated by a comma
my_words = ['flu', 'influenza']   # Each item is separated by a comma
language = 'english'              # Can take values 'english', 'spanish' (all lowercase)

################################################################################
# No need to edit anything below here
################################################################################

# Creating the pattern of filenames based on years to match
years = '|'
years = years.join(year_list)
pattern = '(' + years + ')([0-9]{4})*'
date_pattern = re.compile(pattern)

# Location of files with text for a day's paper
volume_path = datapath + title + '/volumes/'
my_volumes = os.listdir(volume_path)

# Use date pattern from above to restrict to dates of interest
my_volumes = list(filter(date_pattern.match, my_volumes))

# Sort them for easier bookkeeping
my_volumes.sort()

# Create a table that will hold the relative frequency for each date
dates = []
for one_file in my_volumes:
    one_date = str(one_file[0:4]) + '-' + str(one_file[4:6]) + '-' + str(one_file[6:8])
    dates.append(one_date)

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ['Date'])

# Set all frequencies to zero
results_table['Frequency'] = 0.0

# Cycle over all issues and do relative frequency calculations
for issue in my_volumes:
    issue_text = dc.CleanText(filename = volume_path + issue, language = language)
    issue_text = issue_text.clean_list
    
    # Create a table with words
    word_table = pandas.Series(issue_text)

    # Calculate relative frequencies of all words in the issue
    word_freqs = word_table.value_counts(normalize = True)
    
    # Pull out only values that match words of interest
    my_freqs = word_freqs.filter(my_words)
    
    # Get the total frequency for words of interest
    total_my_freq = my_freqs.sum()
    
    # Format the date from the name of the file so we know where to put
    # the data in our table
    issue_date = str(issue[0:4]) + '-' + str(issue[4:6]) + '-' + str(issue[6:8])
    
    # Add the date & relative frequency to our data table
    results_table.loc[results_table['Date'] == issue_date, 'Frequency'] = total_my_freq
    
# Analyses are all done, plot the figure
my_figure = px.line(results_table, x = 'Date', y = 'Frequency')
my_figure.show()

### Una sola palabra en dos periódicos
El siguiente código está diseñado para analizar un conjunto de palabras para un par de títulos de periódicos individuales. Tal como está escrito, el bloque de código observará la frecuencia de palabras relacionadas con Alemania en _El Tucsonense_ y _The Bisbee Daily Review_ durante los años 1917-1919. Tenga en cuenta que debido a que estos documentos son en diferentes idiomas, debemos proporcionar conjuntos de palabras apropiados para cada uno de los documentos.

Actualice los valores correspondientes de títulos, palabras e idiomas para las palabras y títulos de interés. Tenga en cuenta que cuanto más largo sea el intervalo de tiempo que está observando, más tiempo puede llevar el análisis. Cuando el análisis haya finalizado, el asterisco entre corchetes en la parte inferior izquierda se reemplazará con un número (es decir, `In [*]` becomes something like `In [6]`) y el gráfico se imprimirá debajo del bloque de código.

In [ ]:
# Code for one set of words, two newspapers

# Change these to directories of the titles of interest, be sure to use lowercase 
# and no spaces (it may be easiest to copy & paste from the "directory" 
# column in the table above)
title_1 = 'el-tucsonense'
title_2 = 'bisbee-daily-review' 

# The "human readable" names of the newspaper titles that will show up on the 
# plot
title_1_name = 'El Tucsonense'
title_2_name = 'Bisbee Daily Review'

# List the years of interest, each enclosed in quotation marks (') and separated
# by commas
year_list = ['1917', '1918', '1919']

# What words are you interested in? You can add as many as you like, 
# just be sure to enclose each in quotation marks (') and separate with a comma
# Also, keep them lower case, even if they are proper nouns
words_1 = ['alemania', 'alemana', 'alemán'] 
words_2 = ['germany', 'german']

# Specify the language of the title you are looking at (all lowercase)
# Possible values: english, spanish, arabic, turkish, etc.
language_1 = 'spanish'
language_2 = 'english'

################################################################################
# No need to edit anything below here
################################################################################

# Creating the pattern of filenames based on years to match
years = '|'
years = years.join(year_list)
pattern = '(' + years + ')([0-9]{4})*'
date_pattern = re.compile(pattern)

# Create dictionary with information about each title, for easier
# iteration
title_data = {}
title_data[title_1] = {
    'directory' : title_1,
    'name' : title_1_name,
    'words' : words_1,
    'language' : language_1,
    'volume_path' : datapath + title_1 + '/volumes/'
}
title_data[title_1]['volumes'] = os.listdir(title_data[title_1]['volume_path'])
title_data[title_2] = {
    'directory' : title_2,
    'name' : title_2_name,
    'words' : words_2,
    'language' : language_2,
    'volume_path' : datapath + title_2 + '/volumes/'
}
title_data[title_2]['volumes'] = os.listdir(title_data[title_2]['volume_path'])

# Find out all the dates of papers we are looking at
dates = []
for one_file in (title_data[title_1]['volumes'] + title_data[title_2]['volumes']):
    one_date = str(one_file[0:4]) + '-' + str(one_file[4:6]) + '-' + str(one_file[6:8])
    # Only add unique values to avoid duplication
    if one_date not in dates:
        dates.append(one_date)
dates.sort()

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ['Date'])

# Set all frequencies to None
results_table[title_1_name] = None
results_table[title_2_name] = None

# Cycle over each title
for title in [title_1, title_2]:
    title_directory = title_data[title]['directory'] # string
    title_name = title_data[title]['name']           # string
    words = title_data[title]['words']               # list
    language = title_data[title]['language']         # string
    volume_path = title_data[title]['volume_path']   # string
    volumes = title_data[title]['volumes']           # list

    # List of volumes
#     volume_path = title_data[title]['volume_path']   # string
#     volumes = os.listdir(volume_path)
    
    # Use date pattern from above to restrict to dates of interest
    volumes = list(filter(date_pattern.match, volumes))

    # Sort them for easier bookkeeping
    volumes.sort()
    
    # Cycle over all the issues of the current title
    printed = False
    for issue in volumes:
        issue_text = dc.CleanText(filename = volume_path + issue, language = language)
        
        # Clean the text (remove stop words, punctuation, etc.)
        issue_text = issue_text.clean_list
        
        # Create a table with all words from the issue
        word_table = pandas.Series(issue_text)
        
        # Calculate relative frequencies of all words in the issue
        word_freqs = word_table.value_counts(normalize = True)
        
        # Pull out only values that match words of interest
        words_freqs = word_freqs.filter(words)

        # Get the total frequency for words of interest
        total_word_freq = words_freqs.sum()

        # Format the date from the name of the file so we know where to put
        # the data in our table
        issue_date = str(issue[0:4]) + '-' + str(issue[4:6]) + '-' + str(issue[6:8])

        # Add the date & relative frequency to our data table
        results_table.loc[results_table['Date'] == issue_date, title_name] = total_word_freq

# Analyses are all done, but we need to transform data to "long" format
results_melt = results_table.melt(id_vars = 'Date', value_vars = [title_1_name, title_2_name])

# By default, two columns created are called "value" and "variable", we want 
# to rename them
results_melt.rename(columns = {'value':'Frequency', 'variable':'Title'}, inplace = True)

# Before plotting, remove rows with missing values
results_clean = results_melt.dropna()

# plot the figure
my_figure = px.line(results_clean, x = 'Date' , y = 'Frequency' , color = 'Title')
my_figure.show()

### Dos palabras en un periódico
El siguiente código está diseñado para analizar dos conjuntos de palabras para un título de periódico individual. Tal como está escrito, el bloque de código observará la frecuencia de palabras relacionadas con Alemania y las relacionadas con Japón en _El Tucsonense_ durante los años 1917-1919.

Actualice los valores correspondientes de título, palabras e idioma para las palabras y el título de interés. Tenga en cuenta que cuanto más largo sea el intervalo de tiempo que está observando, más tiempo puede llevar el análisis. Cuando el análisis haya finalizado, el asterisco entre corchetes en la parte inferior izquierda se reemplazará con un número (es decir, `In [*]` se convierte en algo como `In [6]`) y el gráfico se imprimirá debajo del bloque de código.

In [ ]:
# Change this to directory of the title of interest, be sure to use lowercase 
# and no spaces (it may be easiest to copy & paste from the "directory" 
# column in the table above)
title = 'el-tucsonense' 

# List the years of interest, each enclosed in quotation marks (') and separated
# by commas
year_list = ['1917', '1918', '1919']

# What words are you interested in? You can add as many as you like, 
# just be sure to enclose each in quotation marks (') and separate with a comma
# Also, keep them lower case, even if they are proper nouns
words_1 = ['alemania', 'alemana', 'alemán']
words_1_name = 'Germany'
words_2 = ['japona', 'japón']
words_2_name = 'Japan'

# Specify the language of the title you are looking at (all lowercase)
# Possible values: english, spanish, arabic, turkish, etc.
language = 'spanish'

################################################################################
# No need to edit anything below here
################################################################################

# Creating the pattern of filenames based on years to match
years = '|'
years = years.join(year_list)
pattern = '(' + years + ')([0-9]{4})*'
date_pattern = re.compile(pattern)


# Location of files with text for a day's paper
volume_path = datapath + title + '/volumes/'
my_volumes = os.listdir(volume_path)

# Use date pattern from above to restrict to dates of interest
my_volumes = list(filter(date_pattern.match, my_volumes))

# Sort them for easier bookkeeping
my_volumes.sort()

# Create a table that will hold the relative frequency for each date
dates = []
for one_file in my_volumes:
    one_date = str(one_file[0:4]) + '-' + str(one_file[4:6]) + '-' + str(one_file[6:8])
    dates.append(one_date)

# Add those dates to a data frame
results_table = pandas.DataFrame(dates, columns = ['Date'])

# Set all frequencies to zero
results_table[words_1_name] = 0.0
results_table[words_2_name] = 0.0

# Cycle over all issues and do relative frequency calculations
for issue in my_volumes:
    issue_text = dc.CleanText(filename = volume_path + issue, language = language)
    issue_text = issue_text.clean_list
    
    # Create a table with words
    word_table = pandas.Series(issue_text)

    # Calculate relative frequencies of all words in the issue
    word_freqs = word_table.value_counts(normalize = True)
    
    # Pull out only values that match words of interest
    words_1_freqs = word_freqs.filter(words_1)
    words_2_freqs = word_freqs.filter(words_2)

    # Get the total frequency for words of interest
    total_words_1 = words_1_freqs.sum()
    total_words_2 = words_2_freqs.sum()
    
    # Format the date from the name of the file so we know where to put
    # the data in our table
    issue_date = str(issue[0:4]) + "-" + str(issue[4:6]) + "-" + str(issue[6:8])
    
    # Add the date & relative frequency to our data table
    results_table.loc[results_table['Date'] == issue_date, words_1_name] = total_words_1
    results_table.loc[results_table['Date'] == issue_date, words_2_name] = total_words_2
    
# Analyses are all done, but we need to transform data to "long" format
results_melt = results_table.melt(id_vars = 'Date', value_vars = [words_1_name, words_2_name])

# By default, two columns created are called "value" and "variable", we want 
# to rename them
results_melt.rename(columns = {'value':'Frequency', 'variable':'Words'}, inplace = True)

# plot the figure
my_figure = px.line(results_melt, x = 'Date' , y = 'Frequency' , color = 'Words')
my_figure.show()

[![](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)](https://creativecommons.org/licenses/by/4.0/legalcode)
Esta lección tiene licencia bajo [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/legalcode) 2020 a Jeffrey C. Oliver.